# 0- Setting the folders

In [48]:
import os

In [49]:
API_DIRECTORY_PATH = './NYT_API_RAW/'
PARSED_DIRECTORY_PATH = './NYT_API_clean/'

if os.path.exists(API_DIRECTORY_PATH):
    print("Raw NYT data folder exist")
else:
    print("Raw NYT data created")
    os.makedirs(API_DIRECTORY_PATH)


Raw NYT data folder exist
Clean NYT data folder exist


# 1- Archive API request

In [53]:
import requests
import pandas as pd
import time 
import numpy as np
import os

#Progress bar for pandas
from tqdm import tqdm
tqdm.pandas()

from pandas import json_normalize

NYT_API_KEY = 'YOUR API KEY'

In [64]:
import os
import time
import requests
import pandas as pd


def get_nyt_archive_api(year_start=1980, year_end=2023, directory_path=API_DIRECTORY_PATH):
    """
    Function to get the data from the NYT Archive API
    """
    for year in range(year_start, year_end+1):
        nyt_data_year_path = os.path.join(directory_path, str(year))
        
        if not os.path.isdir(nyt_data_year_path):
            os.makedirs(nyt_data_year_path)

        for month in range(1, 13):           
            nyt_data_year_month_file = os.path.join(nyt_data_year_path, f"NYT_Archive_API_{year}_{month}.csv")
            
            try:
                monthly_data = pd.read_csv(nyt_data_year_month_file)
                continue
            except:
                print(f'File empty or does not exist: {nyt_data_year_month_file}')

                time.sleep(15) # To stay within API limits
                
                try:
                    nyt_archive = requests.get(f'https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={NYT_API_KEY}')

                    if nyt_archive.status_code != 200:
                        print(f"Request failed with status code {nyt_archive.status_code} for {month}-{year}")
                        break

                    nyt_df = pd.DataFrame.from_dict(nyt_archive.json()['response']['docs'])
                    nyt_df.reset_index(drop=True, inplace=True)
                    nyt_df.to_csv(nyt_data_year_month_file, index=False)

                    print(f"{year} - {month} - {len(nyt_archive.json()['response']['docs'])}" + " - Export finished")

                except requests.exceptions.RequestException as e:
                    print(f"{month} - {year} - FAIL: {e}")


In [70]:
#Give the year to download the data from:

year_start = 1853
year_end = 2023

get_nyt_archive_api(year_start, year_end, API_DIRECTORY_PATH)

File empty or does not exist: ./NYT_API_RAW/1978/NYT_Archive_API_1978_9.csv
1978 - 9 - 0 - Export finished
File empty or does not exist: ./NYT_API_RAW/1978/NYT_Archive_API_1978_10.csv
1978 - 10 - 0 - Export finished
File empty or does not exist: ./NYT_API_RAW/2023/NYT_Archive_API_2023_7.csv
Request failed with status code 403 for 7-2023
